# 2-11. 프로젝트

## 2-11. 프로젝트 (1) load_digits : 손글씨를 분류해 봅시다

### 데이터 준비
- digits 데이터는 총 1,797개의 샘플로 구성되어 있음
- 각 데이터는 64개의 특징 값을 가지고 있음
    - 8x8 이미지로 각 픽셀은 0에서 16사이의 값을 가지고 있음
    - 각각의 픽셀값은 동일한 범위에 놓여지도록 정규화가 적용되어 있음
- 클래스는 0부터 9까지 총 10가지

In [1]:
from sklearn.datasets import load_digits


data_digits = load_digits()

feat = data_digits.data
label = data_digits.target
# print(data_digits.feature_names)
print(feat[0])
print(feat[0].shape)
print(data_digits.target_names)
print(data_digits.DESCR)


[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
 15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
  0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
  0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
(64,)
[0 1 2 3 4 5 6 7 8 9]
.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 1797
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were

- 데이터를 훈련과 테스트용으로 나누었고, train_test_split 함수에서 정의된 기본 비율 0.75:0.25를 그대로 사용하였음
- 여러가지 모델이 사용되지만, 사용방법은 같기 때문에 자료를 입력받아 결과를 출력하는 함수 선언
  - 필요에 따라 데이터 스케일링을 적용

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, LogisticRegression

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# 코드 재생산성을 위해 시드 지정
seed = 1234
models = {
    "DecisionTreeClassifier": DecisionTreeClassifier,
    "RandomForestClassifier": RandomForestClassifier,
    "SVM": SVC,
    "SGDClassifier": SGDClassifier,
    "LogisticRegression": LogisticRegression,
}


def get_model_result(name, model_init, feat, label, scale=False):
    x_train, x_test, y_train, y_test = train_test_split(feat, label, random_state=seed)
    scaler = StandardScaler()
    if scale:
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
    model = model_init(random_state=seed)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print()
    print(f"==== Classification Report of {name} ====")
    print(classification_report(y_pred, y_test))
    print(f"==== Confusion Matrix of {name} ====")
    print(confusion_matrix(y_pred, y_test))


### Decision Tree

In [3]:
get_model_result("Decision Tree", DecisionTreeClassifier, feat, label)


==== Classification Report of Decision Tree ====
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        37
           1       0.89      0.81      0.85        52
           2       0.78      0.86      0.82        44
           3       0.82      0.77      0.79        47
           4       0.87      0.92      0.89        49
           5       0.86      0.91      0.88        46
           6       0.96      0.86      0.91        50
           7       0.97      0.81      0.89        48
           8       0.72      0.82      0.77        38
           9       0.77      0.87      0.82        39

    accuracy                           0.86       450
   macro avg       0.86      0.86      0.86       450
weighted avg       0.86      0.86      0.86       450

==== Confusion Matrix of Decision Tree ====
[[36  0  0  0  0  0  0  0  0  1]
 [ 0 42  3  0  2  0  1  0  3  1]
 [ 0  1 38  2  1  0  0  0  1  1]
 [ 0  0  7 36  0  0  0  1  3  0]
 [ 0  0  0  0 4

#### Decision Tree의 결과 분석

- accuracy는 86%
- 0, 1, 4, 6, 7은 평균보다 높은 precision
  - 해당 클래스로 예측한 것 중 정답인 비율을 나타내는 precision이 높다
    - 해당 숫자로 예측했다면 정답일 확률이 높다는 의미
  - confusion matrix에서 0으로 예측한 경우를 살펴보면, 총 37개 예측이 있었고 36개 정답
- 0, 4, 5, 9는 평균보다 높은 recall
  - 실제 클래스에 해당하는 데이터에 대해 예측이 성공한 비율을 나타내는 recall
  - confusion matrix에서 4가 정답인 경우를 살펴보면, 총 49개의 정답 중 45개의 예측이 성공 
- 각 클래스별 precision과 recall의 편차가 크고, 최솟값의 경우 0.72로 낮아 분류 성능이 낮음

### RandomForest

In [4]:
get_model_result("Random Forest", RandomForestClassifier, feat, label)


==== Classification Report of Random Forest ====
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        39
           1       1.00      0.90      0.95        52
           2       1.00      1.00      1.00        49
           3       0.95      1.00      0.98        42
           4       0.94      0.98      0.96        50
           5       0.96      1.00      0.98        47
           6       0.96      0.98      0.97        44
           7       1.00      0.89      0.94        45
           8       0.93      0.95      0.94        42
           9       0.91      1.00      0.95        40

    accuracy                           0.96       450
   macro avg       0.97      0.97      0.96       450
weighted avg       0.97      0.96      0.96       450

==== Confusion Matrix of Random Forest ====
[[37  0  0  1  1  0  0  0  0  0]
 [ 0 47  0  1  0  0  1  0  3  0]
 [ 0  0 49  0  0  0  0  0  0  0]
 [ 0  0  0 42  0  0  0  0  0  0]
 [ 0  0  0  0 4

#### Random Forest의 결과 분석

- accuracy는 96%
- precision은 최소 0.91이상, recall은 0.89이상으로 높은 수치를 보여 분류 성능이 우수하다고 판단

### SVM

In [5]:
get_model_result("SVM", SVC, feat, label)


==== Classification Report of SVM ====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37
           1       1.00      0.98      0.99        48
           2       1.00      1.00      1.00        49
           3       1.00      1.00      1.00        44
           4       0.98      1.00      0.99        51
           5       0.98      1.00      0.99        48
           6       1.00      0.98      0.99        46
           7       1.00      0.98      0.99        41
           8       0.98      0.95      0.97        44
           9       0.95      1.00      0.98        42

    accuracy                           0.99       450
   macro avg       0.99      0.99      0.99       450
weighted avg       0.99      0.99      0.99       450

==== Confusion Matrix of SVM ====
[[37  0  0  0  0  0  0  0  0  0]
 [ 0 47  0  0  0  0  0  0  1  0]
 [ 0  0 49  0  0  0  0  0  0  0]
 [ 0  0  0 44  0  0  0  0  0  0]
 [ 0  0  0  0 51  0  0  0  0  0]
 [

#### SVM의 결과 분석

- accuracy는 99%
- precision은 최소 0.95이상, recall은 0.95이상으로 높은 수치를 보여 분류 성능이 우수하다고 판단

### SGD Classifier

In [6]:
get_model_result("SGD", SGDClassifier, feat, label)


==== Classification Report of SGD ====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37
           1       0.98      0.92      0.95        50
           2       0.98      0.98      0.98        49
           3       0.98      0.96      0.97        45
           4       0.94      1.00      0.97        49
           5       0.96      0.98      0.97        48
           6       0.96      0.98      0.97        44
           7       1.00      0.93      0.96        43
           8       0.79      0.89      0.84        38
           9       0.93      0.87      0.90        47

    accuracy                           0.95       450
   macro avg       0.95      0.95      0.95       450
weighted avg       0.95      0.95      0.95       450

==== Confusion Matrix of SGD ====
[[37  0  0  0  0  0  0  0  0  0]
 [ 0 46  0  0  1  0  1  0  2  0]
 [ 0  0 48  0  0  0  0  0  1  0]
 [ 0  0  1 43  0  0  0  0  1  0]
 [ 0  0  0  0 49  0  0  0  0  0]
 [

#### SGD Classifier의 결과 분석

- accuracy는 95%
- precision의 경우 평균은 0.95로 높지만, 최솟값의 경우 0.79로 낮음
- recall의 경우 평균이 0.95로 높고, 최솟값의 경우 0.87로 높은 수치
- 특정 클래스에 대한 성능 문제가 발생할 수 있어 다른 모델과 비교하여 분류가 안정적이지 못함

### Logistic Regression

In [7]:
get_model_result("Logistic Regression", LogisticRegression, feat, label)


==== Classification Report of Logistic Regression ====
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        37
           1       1.00      0.94      0.97        50
           2       0.98      0.92      0.95        52
           3       0.95      0.98      0.97        43
           4       0.92      0.98      0.95        49
           5       0.94      1.00      0.97        46
           6       0.98      0.98      0.98        45
           7       1.00      0.98      0.99        41
           8       0.93      0.91      0.92        44
           9       0.98      1.00      0.99        43

    accuracy                           0.96       450
   macro avg       0.97      0.97      0.97       450
weighted avg       0.97      0.96      0.96       450

==== Confusion Matrix of Logistic Regression ====
[[36  0  0  0  1  0  0  0  0  0]
 [ 0 47  1  0  1  0  0  0  1  0]
 [ 1  0 48  1  0  0  0  0  2  0]
 [ 0  0  0 42  0  1  0  0  0  0]
 [ 

C:\Users\jhoon\miniconda3\envs\aiffel\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Logistic Regression의 결과 분석

- accuracy는 96%
- precision은 최소 0.92이상, recall은 0.91이상으로 높은 수치를 보여 분류 성능이 우수하다고 판단
- 모델에 사용되는 파라미터들을 지정하지 않고 기본값으로 사용하는 경우, 지정되어 있는 최대 iteration에 도달하여 수렴하지 못했다는 경고 발생
  - 이러한 경우 iteration 제한을 높이거나 데이터 스케일링을 적용할 수 있지만, 현재 분류 작업에서 우수한 성능을 보여 적용하지 않음

### 결론

- 86%의 정확도를 보인 Decision Tree를 제외한 나머지 RandomForest, SVM, SGD Classifier, Logistic Regression은 95% 이상의 좋은 분류 성능을 보임

## 2-12. 프로젝트 (2) load_wine : 와인을 분류해 봅시다

### 데이터 준비
- wine 데이터는 총 178개의 샘플로 구성
- 각 데이터는 13개의 특징 값을 가지고 있음
- 클래스는 3가지
- 각 특징별로 평균값의 차이가 크고 값의 범위가 크게 달라 정규화 적용

In [8]:
from sklearn.datasets import load_wine


data_wine = load_wine()
feat = data_wine.data
label = data_wine.target
print(data_wine.feature_names)
print(feat[0])
print(feat[0].shape)
print(data_wine.target_names)
print(data_wine.DESCR)


['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
(13,)
['class_0' 'class_1' 'class_2']
.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ====

### Decision Tree

In [9]:
get_model_result("Decision Tree", DecisionTreeClassifier, feat, label, scale=True)


==== Classification Report of Decision Tree ====
              precision    recall  f1-score   support

           0       0.79      1.00      0.88        11
           1       0.84      0.84      0.84        19
           2       1.00      0.80      0.89        15

    accuracy                           0.87        45
   macro avg       0.88      0.88      0.87        45
weighted avg       0.88      0.87      0.87        45

==== Confusion Matrix of Decision Tree ====
[[11  0  0]
 [ 3 16  0]
 [ 0  3 12]]


#### Decision Tree의 결과 분석

- accuracy는 87%
- 특정 클래스에 대해 precision 또는 recall이 1.00인 경우가 있지만, precision의 경우 최소 0.79, recall의 경우 최소 0.80으로 높지 않음
- accuracy와 precision, recall이 0.9 이하로 분류 성능이 뛰어나다고 할 수 없음

### RandomForest

In [10]:
get_model_result("Random Forest", RandomForestClassifier, feat, label, scale=True)


==== Classification Report of Random Forest ====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.89      1.00      0.94        17
           2       1.00      0.86      0.92        14

    accuracy                           0.96        45
   macro avg       0.96      0.95      0.96        45
weighted avg       0.96      0.96      0.96        45

==== Confusion Matrix of Random Forest ====
[[14  0  0]
 [ 0 17  0]
 [ 0  2 12]]


#### Random Forest의 결과 분석

- accuracy는 96%
- precision은 최소 0.89이상, recall은 0.95이상으로 높은 수치를 보여 분류 성능이 우수하다고 판단
  - 첫번째 클래스의 경우 precision과 recall이 모두 1.00 도달

### SVM

In [11]:
get_model_result("SVM", SVC, feat, label, scale=True)



==== Classification Report of SVM ====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.95      1.00      0.97        18
           2       1.00      0.92      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.98        45
weighted avg       0.98      0.98      0.98        45

==== Confusion Matrix of SVM ====
[[14  0  0]
 [ 0 18  0]
 [ 0  1 12]]


#### SVM의 결과 분석

- accuracy는 98%
- precision은 최소 0.95이상, recall은 0.92이상으로 높은 수치를 보여 분류 성능이 우수하다고 판단

### SGD Classifier

In [12]:
get_model_result("SGD", SGDClassifier, feat, label, scale=True)



==== Classification Report of SGD ====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.95      1.00      0.97        18
           2       1.00      0.92      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.98        45
weighted avg       0.98      0.98      0.98        45

==== Confusion Matrix of SGD ====
[[14  0  0]
 [ 0 18  0]
 [ 0  1 12]]


#### SGD Classifier의 결과 분석

- accuracy는 98%
- precision은 최소 0.95이상, recall은 0.92이상으로 높은 수치를 보여 분류 성능이 우수하다고 판단

### Logistic Regression

In [13]:
get_model_result("Logistic Regression", LogisticRegression, feat, label, scale=True)



==== Classification Report of Logistic Regression ====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.95      1.00      0.97        18
           2       1.00      0.92      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.98        45
weighted avg       0.98      0.98      0.98        45

==== Confusion Matrix of Logistic Regression ====
[[14  0  0]
 [ 0 18  0]
 [ 0  1 12]]


#### Logistic Regression의 결과 분석

- accuracy는 98%
- precision은 최소 0.95이상, recall은 0.92이상으로 높은 수치를 보여 분류 성능이 우수하다고 판단

### 결론

- 88%의 정확도를 달성한 Decision Tree를 제외한 나머지 4개의 클래스에서는 96% 이상의 정확도 달성

## 2-13. 프로젝트 (3) load_breast_cancer : 유방암 여부를 진단해 봅시다


### 데이터 준비

- breast cancer 데이터는 총 569개의 샘플로 구성
- 각 데이터는 30개의 특징 값을 가지고 있음
- 클래스는 악성(malignant)과 양성(benign), 총 2가지
- 각 특징별로 평균값의 차이가 크고 값의 범위가 달라 정규화 적용

In [14]:
from sklearn.datasets import load_breast_cancer

data_cancer = load_breast_cancer()
feat = data_cancer.data
label = data_cancer.target
print(data_cancer.feature_names)
print(feat[0])
print(feat[0].shape)
print(data_cancer.target_names)
print(data_cancer.DESCR)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
(30,)
['malignant' 'benign']
.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
-------------------------------------

### Decision Tree

In [15]:
get_model_result("Decision Tree", DecisionTreeClassifier, feat, label, scale=True)



==== Classification Report of Decision Tree ====
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        56
           1       0.94      0.95      0.95        87

    accuracy                           0.94       143
   macro avg       0.94      0.93      0.93       143
weighted avg       0.94      0.94      0.94       143

==== Confusion Matrix of Decision Tree ====
[[51  5]
 [ 4 83]]


#### Decision Tree의 결과 분석

- accuracy는 94%
- 첫번째 클래스인 악성에 대한 recall이 0.91
  - 악성을 놓치는 경우가 발생할 확률이 낮지 않아 유방암 여부 진단에는 적합하지 않다고 판단됨

### Random Forest

In [16]:
get_model_result("Random Forest", RandomForestClassifier, feat, label, scale=True)


==== Classification Report of Random Forest ====
              precision    recall  f1-score   support

           0       0.85      0.96      0.90        49
           1       0.98      0.91      0.95        94

    accuracy                           0.93       143
   macro avg       0.92      0.94      0.92       143
weighted avg       0.94      0.93      0.93       143

==== Confusion Matrix of Random Forest ====
[[47  2]
 [ 8 86]]


#### Random Forest의 결과 분석

- accuracy는 93%
- 첫번째 클래스인 악성에 대한 recall이 0.96
  - precision이 0.85이지만, 악성을 놓치는 경우가 적어 유방암 여부 진단에 적합한 방법이라고 판단됨

### SVM

In [17]:
get_model_result("SVM", SVC, feat, label, scale=True)


==== Classification Report of SVM ====
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        49
           1       1.00      0.94      0.97        94

    accuracy                           0.96       143
   macro avg       0.95      0.97      0.95       143
weighted avg       0.96      0.96      0.96       143

==== Confusion Matrix of SVM ====
[[49  0]
 [ 6 88]]


#### SVM의 결과 분석

- accuracy는 96%
- 첫번째 클래스인 악성에 대한 recall이 1.00
  - precision이 0.89이지만, 악성을 놓치는 경우가 없어 유방암 여부 진단에 좋은 방법이라고 판단됨

### SGD Classifier

In [18]:
get_model_result("SGD", SGDClassifier, feat, label, scale=True)


==== Classification Report of SGD ====
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        52
           1       0.99      0.96      0.97        91

    accuracy                           0.97       143
   macro avg       0.96      0.97      0.96       143
weighted avg       0.97      0.97      0.97       143

==== Confusion Matrix of SGD ====
[[51  1]
 [ 4 87]]


#### SGD Classifier의 결과 분석

- accuracy는 97%
- 첫번째 클래스인 악성에 대한 recall이 0.98
  - precision이 0.93이면서, 악성을 놓치는 경우가 적어 유방암 여부 진단에 좋은 방법이라고 판단됨

### Logistic Regression

In [19]:
get_model_result("Logistic Regression", LogisticRegression, feat, label, scale=True)


==== Classification Report of Logistic Regression ====
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        50
           1       1.00      0.95      0.97        93

    accuracy                           0.97       143
   macro avg       0.95      0.97      0.96       143
weighted avg       0.97      0.97      0.97       143

==== Confusion Matrix of Logistic Regression ====
[[50  0]
 [ 5 88]]


#### Logistic Regression의 결과 분석

- accuracy는 97%
- 첫번째 클래스인 악성에 대한 recall이 1.00
  - precision이 0.91이면서, 악성을 놓치는 경우가 없어 유방암 여부 진단에 좋은 방법이라고 판단됨

### 결론
- Decision Tree의 경우 정확도는 0.94로 높은 편이나, 악성 클래스에 대한 recall이 0.91로 다소 낮아 유방암 진단에는 적합하지 않은 모델이라 판단됨
- SVM, SGD Classifier, Logistic Regression의 경우 0.96의 이상의 정확도를 보이면서, 악성 클래스에 대한 recall 역시 0.98 이상으로 매우 높아 유방암 진단에 적합한 모델이라 판단됨
- Random Forest의 경우, 0.93의 정확도를 보이면서 악성 클래스에 대한 recall이 0.96으로 높은 편이기에 유방암 진단에 사용 가능한 모델이라 판단됨

## 프로젝트 결론
- 3가지 데이터셋에 대해 Decision Tree는 다른 모델에 비교하여 전반적으로 낮은 분류 성능을 보임
  - 이러한 경향은 데이터가 가지는 특징의 수와 무관하게 나타남
- SVM, Logistic Regression은 모든 데이터셋에 대해서 높은 accuracy와 함께 높은 recall을 얻을 수 있었음
  - 기본적인 분류 성능이 뛰어나면서, False Negative가 적은 것이 중요한 데이터셋에서도 좋은 성능을 보일 수 있음


## 루브릭

1. 3가지의 데이터셋의 구성이 합리적으로 진행되었는가? - feature와 label 선정을 위한 데이터 분석과정이 체계적으로 전개됨
2. 3가지 데이터셋에 대해 각각 5가지 모델을 성공적으로 적용하였는가? - 모델학습 및 테스트가 정상적으로 수행되었음
3. 3가지 데이터셋에 대해 모델의 평가지표가 적절히 선택되었는가? - 평가지표 선택 및 이유 설명이 타당함

### 루브릭에 대한 자체평가
1. 데이터셋의 feature의 특성을 분석하여 scale 여부를 결정하여 적용
2. Decision Tree, Random Forest, SVM, SGD Classifier, Logistic Regression의 5가지 모델에 대하여 성공적으로 훈련을 진행하고 성능 평가를 위한 결과를 얻을 수 있었음
3. 3가지 종류의 데이터셋에 대해 accuracy, precision 또는 recall 성능을 개별적으로 분석하였다